In [1]:
# !pip install pythreejs
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [2]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np
import time

In [3]:
def generate_cube(width=1.0, height=1.0, depth=1.0):
    """Generate vertices and faces for a box."""
    vertices = np.array([
        [-width/2, -height/2, -depth/2],  # 0 - Back Bottom Left
        [ width/2, -height/2, -depth/2],  # 1 - Back Bottom Right
        [ width/2,  height/2, -depth/2],  # 2 - Back Top Right
        [-width/2,  height/2, -depth/2],  # 3 - Back Top Left
        [-width/2, -height/2,  depth/2],  # 4 - Front Bottom Left
        [ width/2, -height/2,  depth/2],  # 5 - Front Bottom Right
        [ width/2,  height/2,  depth/2],  # 6 - Front Top Right
        [-width/2,  height/2,  depth/2]   # 7 - Front Top Left
    ], dtype=np.float32)

    # Define 12 triangles (6 faces x 2 triangles per face)
    faces = np.array([
        [4, 5, 6], [4, 6, 7],  # Front (+Z)
        [1, 0, 3], [1, 3, 2],  # Back (-Z)
        [5, 1, 2], [5, 2, 6],  # Right (+X)
        [0, 4, 7], [0, 7, 3],  # Left (-X)
        [3, 7, 6], [3, 6, 2],  # Top (+Y)
        [0, 1, 5], [0, 5, 4]   # Bottom (-Y)
    ], dtype=np.uint32)
    
    # Define normals per triangle face (each normal repeats for 2 triangles per face)
    normals = np.array([
        [-0.577, -0.577, -0.577],  # 0 - Averaged from Back, Bottom, Left
        [ 0.577, -0.577, -0.577],  # 1 - Averaged from Back, Bottom, Right
        [ 0.577,  0.577, -0.577],  # 2 - Averaged from Back, Top, Right
        [-0.577,  0.577, -0.577],  # 3 - Averaged from Back, Top, Left
        [-0.577, -0.577,  0.577],  # 4 - Averaged from Front, Bottom, Left
        [ 0.577, -0.577,  0.577],  # 5 - Averaged from Front, Bottom, Right
        [ 0.577,  0.577,  0.577],  # 6 - Averaged from Front, Top, Right
        [-0.577,  0.577,  0.577]   # 7 - Averaged from Front, Top, Left
    ], dtype=np.float32)
    return vertices, faces, normals

def draw_cube(width=1.0, height=1.0, depth=1.0, color='green', position=[0, 0, 0]):
    """Draw a cube with a given size and color."""
    vertices, faces, normals = generate_cube(width, height, depth)
    geometry = BufferGeometry(
        attributes={
            'position': BufferAttribute(array=vertices.copy(), normalized=False),
            'normal': BufferAttribute(array=normals.copy(), normalized=True),
        },
        index=BufferAttribute(array=faces.copy().ravel(), normalized=False)
    )
    material = MeshStandardMaterial(
        color=color,  # RGB color of the material
        flatShading=True,
        wireframe=True,
    )
    cube = Mesh(geometry=geometry, material=material)
    cube.castShadow = True
    
    cube.geometry.attributes['position'].array += np.array(position, dtype=np.float32)
    return cube

def draw_plane(origin=[0, 0, 0], normal=[0, 1, 0], size=10, color='gray', opacity=0.5):
    """Draw a plane with a given normal and size."""
    # Define the plane geometry
    plane_geometry = PlaneGeometry(width=size, height=size)
    plane_material = MeshStandardMaterial(
        color=color, 
        transparent=True, 
        opacity=opacity, 
        side='DoubleSide')
    plane = Mesh(geometry=plane_geometry, material=plane_material)
    plane.receiveShadow = True
    
    # Rotate the plane to the normal vector
    default_normal = np.array([0, 0, 1])  # PlaneGeometry default normal
    axis = np.cross(default_normal, normal)
    angle = np.arccos(np.dot(default_normal, normal))  # Angle between vectors
    
    if np.linalg.norm(axis) > 1e-6:  # Avoid division by zero if vectors are parallel
        axis = axis / np.linalg.norm(axis)
        plane.quaternion = list(np.append(axis * np.sin(angle / 2), np.cos(angle / 2)))  # Quaternion rotation 
    
    plane.position = origin
    return plane

# World Setup

In [4]:


# Rendering setup
width, height = 800, 600 
aspect = width / height
scene = Scene(background='black')  # Background color of the scene
camera = PerspectiveCamera(
    position=[5, 5, 10], 
    lookAt=[0, 0, 0],
    aspect=aspect
    )

ambient_light = AmbientLight(color='white', intensity=1.0)  # Soft overall light
directional_light = DirectionalLight(
    color='white',
    intensity=2.0,        # Brightness
    position=[5, 5, 0],   # Light direction
    
)
spot_light = SpotLight(
    color='white',
    intensity=5.0,        # Increase brightness
    position=[0, 5, 0],   # Place the light above the cube
    distance=10,          # Light range (covers cube)
    angle=0.5,            # Cone angle (medium focus)
    penumbra=0.3,         # Soft shadow edges
    decay=2,              # Realistic falloff
)
spot_light.castShadow = True  # Enable shadow casting
scene.add([ambient_light, spot_light, directional_light])

controller = OrbitControls(controlling=camera, enableRotate=True)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'


# Scene setup
"""Draw a cube with a given size and color."""
vertices, faces, normals = generate_cube(1, 1, 1)
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(array=vertices.copy(), normalized=False),
        'normal': BufferAttribute(array=normals.copy(), normalized=True),
    },
    index=BufferAttribute(array=faces.copy().ravel(), normalized=False)
)
material = MeshStandardMaterial(
    color='green',  # RGB color of the material
    flatShading=True,
    wireframe=True,
)
cube1 = Mesh(geometry=geometry, material=material)
cube1.castShadow = True

cube1.geometry.attributes['position'].array += np.array([0, 1, 0], dtype=np.float32)
# cube1 = draw_cube(width=1, height=1, depth=1, color='green', position=[0, 2, 0])
# cube2 = draw_cube(width=1, height=1, depth=1, color='blue', position=[1.5, 1, 0])

scene.add(cube1)

plane = draw_plane(normal=[0, 1, 0], size=10, color='gray', opacity=0.5)
scene.add(plane)

button = widgets.Button(description="RESET")
output = widgets.Output()
def on_button_clicked(b):
    cube1.geometry.attributes['position'].array = vertices.copy() + np.array([0, 1, 0], dtype=np.float32)

button.on_click(on_button_clicked)


display(widgets.VBox([renderer, button, output]))

In [ ]:
def solve_volume_constraint(x1, x2, x3, x4, w1, w2, w3, w4, rest_volume, compliance, h, prev_lambda=None):
    # C(x1, x2, x3, x4) = (1/6) * |(x2 - x1) x (x3 - x1)| - rest_volume
    n = np.cross(x2 - x1, x3 - x1)
    d = np.linalg.norm(n)
    C = (1/6) * d - rest_volume
    
    if d < 1e-6:
        return 0, 0, 0, 0, 0
    
    n /= d
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + w3 + w4 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = w2 * dx
    dx3 = w3 * dx
    dx4 = w4 * dx
    
    return dx1, dx2, dx3, dx4, new_lambda

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda

def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):
    compliance = 0
    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    # if abs(ray_projection) < 1e-13:
    #     return 0, None, None
    
    t = - ray_start / ray_projection
    
    # No Collision 
    if t > 1:
        return 0, None, None

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - ray_projection * normal
        C = (x1 - q_s) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None


# Simulation parameters
gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 1
h = time_step/substeps


cube1.currPos = cube1.geometry.attributes['position'].array.copy()
cube1.currVel = np.zeros(cube1.currPos.shape)

faces = cube1.geometry.index.array.reshape(-1, 3)
cube1.edges = set()
for face in faces:
    for i in range(3):
        cube1.edges.add(tuple(sorted([face[i], face[(i+1)%3]])))
    
while True:
    ## Gather Collisions Candidates
    coll_idx = np.where(cube1.currPos<0)[0]
    print(cube1.currPos[:, 1]); exit()
    for _ in range(substeps):
        ## Check Actual Collisions
        
        ## integration
        cube1.prevPos = cube1.currPos.copy()
        cube1.currVel += gravity * h * 1.0
        cube1.currPos += cube1.currVel * h
            
        # Solve Distance Constraints
        for i, j in cube1.edges:
            x1, x2 = cube1.currPos[i], cube1.currPos[j]
            dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=1, compliance=0.0, h=h, prev_lambda=0)
            cube1.currPos[i] += dx1
            cube1.currPos[j] += dx2         

        ## Solve Collisons
        for i in coll_idx:
            x1 = cube1.currPos[i]
            x1_prev = cube1.prevPos[i]
            normal = np.array([0, 1, 0])
            offset = np.array([0, 0, 0])
            dx, _, _ = static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=0)
            cube1.currPos[i] += dx
        
        ## Updata Velocities with solved positions
        cube1.currVel = (cube1.currPos - cube1.prevPos) / h
            
        ## Solve Velocities  - Damping and Friction
        # solve_velocities()
    cube1.geometry.attributes['position'].array = cube1.currPos.copy()
    
    
    time.sleep(0.0333)

[0.5 0.5 1.5 1.5 0.5 0.5 1.5 1.5]
[0.5       0.5       1.3517925 1.401741  0.5       0.5759161 1.4272994
 1.46266  ]
[0.44769722 0.42941755 1.3422269  1.2673184  0.5        0.50998557
 1.3925474  1.3527042 ]
[0.40311313 0.42941755 1.2550708  1.2287722  0.5        0.39228454
 1.3069497  1.0378042 ]
[0.23096198 0.42247218 1.1513411  1.1960963  0.46860683 0.30840796
 1.1735368  0.82630426]
[0.09717929 0.39531597 1.1131493  1.0987618  0.24042518 0.26409182
 1.0662416  0.63967746]


/tmp/ipykernel_1882168/3132088691.py:56: RuntimeWarning: divide by zero encountered in scalar divide
  t = - ray_start / ray_projection


[0.0000000e+00 3.0681032e-01 1.1095934e+00 9.3745160e-01 6.9116720e-04
 2.1784671e-01 9.2697537e-01 5.1074779e-01]


/tmp/ipykernel_1882168/3132088691.py:56: RuntimeWarning: invalid value encountered in scalar divide
  t = - ray_start / ray_projection


TypeError: cannot unpack non-iterable NoneType object

: 

In [ ]:
def get_bounding_box(body):
    min = np.array([np.inf, np.inf, np.inf])
    max = np.array([-np.inf, -np.inf, -np.inf])
    for vertex in body.geometry.attributes['position'].array:
        min = np.minimum(min, vertex)
        max = np.maximum(max, vertex)
    return min, max

def check_collision(body1, body2):
    min1, max1 = get_bounding_box(body1)
    min2, max2 = get_bounding_box(body2)
    return min1[0] <= max2[0] and max1[0] >= min2[0] and min1[1] <= max2[1] and max1[1] >= min2[1] and min1[2] <= max2[2] and max1[2] >= min2[2]

def collect_collisions(bodies, plane):
    collisions = []
    # Check collision body vs body
    for i in range(len(bodies)):
        for j in range(i+1, len(bodies)):
            if check_collision(bodies[i], bodies[j]):
                collisions.append((bodies[i], bodies[j]))
    
    # Check collision body vs plane
    for body in bodies:
        if np.any(np.dot(body.geometry.attributes['position'].array, plane.normal) < 0):
            ground_collisions.append((body, plane))

    return collisions, ground_collisions

def check_contacts(collision, ground_collisions):
    contacts = []
    for body, plane in ground_collisions:
        n, o = plane.normal, plane.origin
            
        v = (body.currPos - body.prevPos)
        t = - (np.dot(n, body.prevPos) + o) / np.dot(n, v)
    
        q_c = body.prevPos + t * v
        q_s = body.prevPos - (np.dot(n, body.prevPos) + o) * n
        
        cont = np.where((0<t) and (t<=1))[0]
        stat = np.where(t<=0)[0]
        
        contacts.extend([(body, i, q_c[i], n, 'continuous') for i in cont])
        contacts.extend([(body, i, q_s[i], n, 'static') for i in stat])     
        
        return contacts
    
def solve_contacts(contacts):
    for (body, i, q, n, coll_type) in contacts:
        if coll_type == 'continuous':
            solve_continuous_contact(body, i, q, n)
        elif coll_type == 'static':
            solve_static_contact(body, i, q, n)
        
    
    def solve_positions(body):
        
        



In [ ]:
# Simulation parameters
gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 5
h = time_step/substeps
while True:
    ## Gather Collisions Candidates
    collisions, ground_collisions = collect_collisions(bodies, plane) # -> the pair of bodies
    
    for _ in range(substeps):
        ## Check Actual Collisions
        contacts = check_contacts(collisions, ground_collisions) # -> constraint generate : 
        
        for body in bodies:
            integrate(body, gravity, h) # update position and orientation with external forces and torques
            
        ## Solve Constraints : distance constraints (attach, hinge, etc.)
        # for constraint in constraints:
        #     constraint.solve(h)
            
        for body in bodies:
            for (x1, x2, rest_length, curr_lambda) in body.distance_constraints:
                solve_distance_constraint(x1, x2, 1, 1, rest_length=rest_length, compliance=0.0, h=h, prev_lambda=curr_lambda)
        ## Solve Collisons
        solve_contacts(contacts)
        
        ## Updata Velocities with solved positions
        for body in bodies:
            update_velocities()
            
        ## Solve Velocities  - Damping and Friction
        solve_velocities()
    
    update_mesh(bodies)
    time.sleep(0.01)

In [ ]:
def dynamic_collision_constraint(x1, x2, w1, w2, n, d, compliance, h, prev_lambda=None):
    pass


def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):
    compliance = 0
    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    
    t = - ray_start / ray_projection
    
    # No Collision 
    if t > 1:
        return 0, None, None

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - ray_projection * normal
        C = (x1 - q_s) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda


